# Goal 
In this notbook I will reorganize the columns for:
1. **Food taxonomy**
2. **Ferment type**
3. **Processing**

## The taxonomy levels:
1. Top level - Animal or Plant origin
2. Second level - Main ingredient type
>> 2.1 Animal will be separated to: <br>
>> >> 2.1.1 - Meat<br>
>> >> 2.1.2 - Fish<br>
>> >> 2.1.3 - Dairy<br>

>> 2.2 Plant will be separated to: <br>
>> >> 2.2.1 - Vegetables/Fruit<br>
>> >> 2.2.2 - Cereal<br>
>> >> 2.2.3 - Legumes<br>
>> >> 2.2.4 - Root<br>
3. Third level - food name. 

## Ferment type:
We will indicate the following:
1. Acid - lactic, acetic, mixed
2. Alcohol - yes, no
3. Degradation - protein, fat, or mixed

While there are foods that contain both we will try to indicate the main product produced. <br> 
For example - the main acid ferment in cheese is lactic acid, but most (if not all) cheese also contian acetic acid.

## Processing:
We will indicate the following:
1. Aerobic vs anaerobic fermentation.
2. Addtional ingredients - salt, enzymes (e.g. renet)
3. Fermentation temp
* For ferementation we will focus on the primary fermentation and not the aging temp.


### Start date: 08/04/25
### Last update to goals: 

In [1]:
import pandas as pd
import numpy as np
import os
import sys

# load the data
dataset = pd.read_csv('data/data_sorce/2025-03-24-all-ff-mag-metadata-cleaned-curated.tsv', sep='\t')

# get the relevant columns
dataset = dataset[['sample_description', 'fermented_food', 'specific_substrate', 'substrate_category', 'general_category']]

# remove duplicates
dataset = dataset.drop_duplicates()
dataset = dataset.reset_index(drop=True)

display(dataset.shape, dataset.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_sorce/2025-03-24-all-ff-mag-metadata-cleaned-curated.tsv'